In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

if torch.cuda.is_available():
    print('GPU is available')
    print('GPU Name:', torch.cuda.get_device_name(0))
    print('GPU Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
else:
    print('GPU is not available')

GPU is available
GPU Name: Tesla T4
GPU Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from google.colab import drive

drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/datasets/train'
val_dir = '/content/drive/MyDrive/datasets/val'
test_dir = '/content/drive/MyDrive/datasets/test'

transform = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = torch.device("cuda")
model = torchvision.models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)


    epoch_loss = running_loss / len(train_dataset)

    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {epoch_loss:.4f}')


    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)


    val_loss = val_loss / len(val_dataset)

    print(f'Val Loss: {val_loss:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = model.state_dict()

    torch.save({'model_state_dict': best_model_wts,
            'class_labels': train_dataset.classes},
           'animals.pth')





Epoch 1/10, Train Loss: 0.5313
Val Loss: 0.5025
Epoch 2/10, Train Loss: 0.2715
Val Loss: 0.3560
Epoch 3/10, Train Loss: 0.2117
Val Loss: 0.3482
Epoch 4/10, Train Loss: 0.1512
Val Loss: 0.3471
Epoch 5/10, Train Loss: 0.1296
Val Loss: 0.3383
Epoch 6/10, Train Loss: 0.1150
Val Loss: 0.3645
Epoch 7/10, Train Loss: 0.0985
Val Loss: 0.3362
Epoch 8/10, Train Loss: 0.0855
Val Loss: 0.3024
Epoch 9/10, Train Loss: 0.0766
Val Loss: 0.3174
Epoch 10/10, Train Loss: 0.0965
Val Loss: 0.3016


In [11]:
from PIL import Image
import os

weights = torch.load('/content/animals.pth')
model_state_dict = weights['model_state_dict']
class_labels = weights['class_labels']

model = models.resnet50(weights=None)
model.load_state_dict(model_state_dict)
model.eval()
model = model.to('cuda')


total_images = 0
correct_predictions = 0

for subdir, _, files in os.walk(test_dir):
    true_class = os.path.basename(subdir)
    if true_class not in class_labels:
        continue

    for file in files:
        if file != '.DS_Store':
            image_path = os.path.abspath(os.path.join(subdir, file))
            print(image_path)
            image = Image.open(image_path)

            if image.mode == 'RGBA':
                image = image.convert('RGB')

            image = transform(image)
            image = image.unsqueeze(0)
            image = image.to('cuda')

            with torch.no_grad():
                outputs = model(image)
                _, predicted = torch.max(outputs, 1)
                index = predicted.item()
                predicted_class = class_labels[index]

            total_images += 1
            if predicted_class == true_class:
                correct_predictions += 1

accuracy = correct_predictions / total_images * 100
print(f'Total images: {total_images}')
print(f'Correctly identified images: {correct_predictions}')
print(f'Accuracy: {accuracy:.2f}%')


/content/drive/MyDrive/datasets/test/horse/OIP-ExX0WQ4QL112iaUo_lTPqQHaEK.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-3H-2tSYH9Ip5z5KQqRG9lAHaLH.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-4qPU4AZycLZlpRyqN3qq4gHaFt.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-IRX10K2iB0W0Vo1w9R95ZgHaGS.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-FVp4CIOl6X_s45q1F83PDwAAAA.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-16pJcfR0UbvSvZztag8eNgHaFj.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-21yxHldI7fdzxEE1ysNPLwHaE8.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-FKT6rkU96t4hLEtORfMI8gHaE7.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-IFddpP4Q5BuMMCEvg0TdngHaLF.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-IC30zNopNor-WnbrDJsqwgHaGO.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-3FFrQVgbuAvG5OnsGrKtiAHaI4.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-Cfj1XYF8aLsUILh4TaYepQHaFT.jpeg
/content/drive/MyDrive/datasets/test/horse/OIP-IY6yf